In [1]:
from utils import time_op
import impmeas as imp

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = imp.formulas.GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    satcount_bdd = time_op(f.expectation)() * 2**n
    print(f"BDD satcount = {satcount_bdd}")
    del f

[00010.0770 ms / 0.0101 s / 0.0002 min] satcount
GPMC satcount = 248
[00082.7005 ms / 0.0827 s / 0.0014 min] parse
[00000.0057 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 248.0


In [6]:
n,clauses_to_variables,k = 17, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [7]:
import impmeas.bdds as bdd 

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    x = "x1"
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")
    del f

[01408.2460 ms / 1.4082 s / 0.0235 min] parse
bdd size = 2525
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f85dde539d0> ===
k=0 d result=0.5239 max increase possible=0.2380 
k=1 d result=0.0931 max increase possible=0.0967 
k=2 d result=0.0647 max increase possible=0.0240 
k=3 d result=0.0210 max increase possible=0.0024 
k=4 d result=0.0023 max increase possible=0.0001 
stopped earlier because cannot improve above cutoff.
current value: 0.7049, can be increased by 0.0001 <= 0.0001.
=== DONE ===
[06414.0983 ms / 6.4141 s / 0.1069 min] blame
[00000.5341 ms / 0.0005 s / 0.0000 min] influence
influence 0.14632, 0.70493 <= blame


In [8]:
solver = imp.formulas.GPMC(cs=20000)
imp.formulas.set_pmc_solver(solver)
f = time_op(imp.Formula.parse)(formula)
x = "x1"
infl = (time_op)(imp.influence)(f, x)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[01433.6789 ms / 1.4337 s / 0.0239 min] parse
[00271.9815 ms / 0.2720 s / 0.0045 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3161 ===
k=0 size of cnf: 3497 d result=0.1463 max increase possible=0.4268 
k=1 size of cnf: 3556 d result=0.1588 max increase possible=0.1787 
k=2 size of cnf: 3611 d result=0.1241 max increase possible=0.0409 
k=3 size of cnf: 3662 d result=0.0362 max increase possible=0.0038 
k=4 size of cnf: 3709 d result=0.0037 max increase possible=0.0001 
k=5 size of cnf: 3752 d result=0.0001 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4692, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[22232.9073 ms / 22.2329 s / 0.3705 min] blame
influence 0.14632, 0.46918 <= blame


In [ ]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00881.8030 ms / 0.8818 s / 0.0147 min] parse
[01480.2790 ms / 1.4803 s / 0.0247 min] blame
0.51475 <= blame
[00012.7113 ms / 0.0127 s / 0.0002 min] influence
influence = 0.19141
[06570.6398 ms / 6.5706 s / 0.1095 min] <lambda>
hammer = 0.03937
[00849.1025 ms / 0.8491 s / 0.0142 min] dominating_cgm
[00699.5080 ms / 0.6995 s / 0.0117 min] rectifying_cgm
